In [1]:
#Add Dependencies

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import pprint
from citipy import citipy
import gmaps
import gmaps.datasets
import scipy.stats as stats
#import folium

## Importing CSV Data



In [2]:
#Import citation data from csv
citation_raw = pd.read_csv("Parking_Citations.csv")
citation_df = citation_raw
citation_df.head()

C:\Users\Sisay\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,1114684152,5/1/2018,1310.0,NaN,NaN,CA,201711.0,NaN,GMC,TR,BK,1035 SERRANO,1,1.0,4000A1,NO EVIDENCE OF REG,50.0,6468879.186,1841441.836
1,1122716066,5/1/2018,1000.0,NaN,NaN,CA,201802.0,NaN,FORD,VN,WH,MOSQUEDA,5M37,2.0,80714,PRIVATE PROPERTY,68.0,99999.000,99999.000
2,1120920986,5/1/2018,1140.0,NaN,NaN,CA,201802.0,NaN,BUIC,PA,GR,66TH ST-FIGUEROA ST,1,1.0,8.06E+07,RED ZONE,93.0,99999.000,99999.000
3,1120920614,5/1/2018,1340.0,NaN,NaN,CA,201307.0,NaN,CHEV,TR,SI,PACIFIC/9TH ST,1,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.000,99999.000
4,1120911186,5/1/2018,1405.0,NaN,NaN,CA,201712.0,NaN,HOND,VN,RE,570 4TH ST,MQ134,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.000,99999.000


In [3]:
#Name of columns 
citation_df.columns.tolist()

['Ticket number',
 'Issue Date',
 'Issue time',
 'Meter Id',
 'Marked Time',
 'RP State Plate',
 'Plate Expiry Date',
 'VIN',
 'Make',
 'Body Style',
 'Color',
 'Location',
 'Route',
 'Agency',
 'Violation code',
 'Violation Description',
 'Fine amount',
 'Latitude',
 'Longitude']

## Cleaning Columns from Dataset

In [4]:
#Drop columns that are not needed
citation_df1 = citation_df.drop(["Meter Id","Marked Time","RP State Plate","Plate Expiry Date","VIN","Location","Route","Agency","Violation code","Body Style", "Violation Description", "Fine amount"], axis = 1)
citation_df1.head()

#Drop Lat and Lon = 99999
citation_df2 = citation_df1[(citation_df1["Latitude"] !=99999.000) & (citation_df1["Longitude"] !=99999.000)]
citation_df2.head()


,Ticket number,Issue Date,Issue time,Make,Color,Latitude,Longitude
0,1114684152,5/1/2018,1310.0,GMC,BK,6468879.186,1841441.836
39,1121020504,5/1/2018,1100.0,CHEV,SI,6487917.292,1830966.052
41,1116268053,5/1/2018,1211.0,SUBA,RE,6443160.675,1921957.696
45,1117163961,5/1/2018,1045.0,TOYO,BK,6401115.821,1887444.261
46,1117163950,5/1/2018,2135.0,TOYO,GY,6397793.249,1886756.686


In [24]:
citation_df2.count()

Ticket number    419378
Issue Date       419378
Issue time       419303
Make             419187
Color            419316
Latitude         419377
Longitude        419377
dtype: int64

In [6]:
#Take all rows where column values are not equal to zero
citation_df3 = citation_df2[(citation_df2[["Ticket number", "Issue Date","Issue time","Make","Color","Latitude","Longitude",]] != 0)]
citation_df3.head()


,Ticket number,Issue Date,Issue time,Make,Color,Latitude,Longitude
0,1114684152,5/1/2018,1310.0,GMC,BK,6468879.186,1841441.836
39,1121020504,5/1/2018,1100.0,CHEV,SI,6487917.292,1830966.052
41,1116268053,5/1/2018,1211.0,SUBA,RE,6443160.675,1921957.696
45,1117163961,5/1/2018,1045.0,TOYO,BK,6401115.821,1887444.261
46,1117163950,5/1/2018,2135.0,TOYO,GY,6397793.249,1886756.686


In [8]:
#Drop all rows with blank cells
citation_df4 = citation_df3.fillna("")


,Ticket number,Issue Date,Issue time,Make,Color,Latitude,Longitude
0,1114684152,5/1/2018,1310,GMC,BK,6.46888e+06,1.84144e+06
39,1121020504,5/1/2018,1100,CHEV,SI,6.48792e+06,1.83097e+06
41,1116268053,5/1/2018,1211,SUBA,RE,6.44316e+06,1.92196e+06
45,1117163961,5/1/2018,1045,TOYO,BK,6.40112e+06,1.88744e+06
46,1117163950,5/1/2018,2135,TOYO,GY,6.39779e+06,1.88676e+06


In [9]:
citation_df4.count()

Ticket number    419378
Issue Date       419378
Issue time       419378
Make             419378
Color            419378
Latitude         419378
Longitude        419378
dtype: int64

In [15]:
citation_df4["Make"].replace("", np.nan, inplace=True)
citation_df4.dropna(subset = ["Make"], inplace = True)

In [16]:
citation_df4.sort_values(["Make"],ascending=True)


,Ticket number,Issue Date,Issue time,Make,Color,Latitude,Longitude
411526,1125434741,7/13/2018,1008,AAUD,BK,6.47364e+06,1.85911e+06
189085,1122851796,6/2/2018,1820,ACRA,GY,6.44206e+06,1.804e+06
189094,1122850201,6/2/2018,1535,ACRA,BK,6.43809e+06,1.80165e+06
483097,1125314746,7/27/2018,1210,ACU,BK,6.47287e+06,1.85974e+06
426593,1125777144,7/17/2018,1015,ACU,BK,6.45491e+06,1.84603e+06
253420,4333706440,6/13/2018,1844,ACUR,SL,6.49014e+06,1.84016e+06
448247,4337032346,7/20/2018,1347,ACUR,BK,6.42572e+06,1.89026e+06
216635,4334220833,6/7/2018,927,ACUR,SL,6.4832e+06,1.83943e+06
50030,4331509033,5/9/2018,1851,ACUR,SL,6.45056e+06,1.85262e+06
25000,4332009010,5/4/2018,1603,ACUR,GN,6.41012e+06,1.89084e+06


In [17]:
citation_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 419187 entries, 0 to 498891
Data columns (total 7 columns):
Ticket number    419187 non-null object
Issue Date       419187 non-null object
Issue time       419187 non-null object
Make             419187 non-null object
Color            419187 non-null object
Latitude         419187 non-null object
Longitude        419187 non-null object
dtypes: object(7)
memory usage: 25.6+ MB


In [23]:
citation_df4["Make"].unique()

array(['GMC', 'CHEV', 'SUBA', 'TOYO', 'CHRY', 'ALFA', 'DODG', 'MAZ',
       'PORS', 'HOND', 'BMW', 'FORD', 'JEEP', 'MERZ', 'NISS', 'SUZU',
       'TOYT', 'KIA', 'VOLK', 'LNDR', 'INFI', 'MAZD', 'FIAT', 'SUZI',
       'ACUR', 'ISU', 'HYUN', 'FRHT', 'AUDI', 'GRUM', 'CADI', 'LEX',
       'LEXS', 'VOLV', 'MNNI', 'MITS', 'STRN', 'SAA', 'OTHR', 'LROV',
       'TSMR', 'TRIU', 'UNK', 'MERC', 'PONT', 'SMRT', 'CHEC', 'LEXU',
       'SATU', 'INF', 'BUIC', 'LINC', 'INTL', 'BENT', 'MASE', 'KAWK',
       'HUMM', 'HD', 'JAGU', 'FERR', 'WINN', 'PTRB', 'KW', 'YAMA', 'PLYM',
       'OLDS', 'LNCI', 'ASTO', 'RV', 'ROLS', 'TESL', 'HUND', 'MASR',
       'JAG', 'AER', 'HNO', 'ROL', 'DATS', 'GEO', 'SPEC', 'MBZ', 'MBNZ',
       'SUB', 'MIHI', 'BOUL', 'AUBU', 'MINI', 'VW', 'KAW', 'KTM', 'SCOI',
       'MERK', 'STLG', 'HNDI', 'HYND', 'SCIO', 'RNOV', 'MERS', 'GENE',
       'BMS', 'PCH', 'RANG', 'SAAB', 'RENA', 'ITAS', 'MACK', 'PACK',
       'DAEW', 'MITZ', 'BENZ', 'SUSK', 'IND', 'DAIH', 'LAND', 'PACE',
       'SAT

In [9]:
#Check the total counts in each category
citation_df4["Make"].value_counts()
#citation_df3["Color"].value_counts()



TOYT    82660
HOND    56686
FORD    42931
NISS    38002
CHEV    34271
BMW     22192
MERZ    20021
HYUN    16435
VOLK    15960
DODG    14541
LEXS    13602
KIA     13306
JEEP    12395
MAZD     9543
AUDI     9322
OTHR     7676
GMC      6897
INFI     6814
SUBA     6468
CHRY     5720
ACUR     5632
TOYO     4674
VOLV     4510
LNDR     3856
MITS     3849
CADI     3708
MNNI     3391
FRHT     2626
UNK      2339
PORS     2335
        ...  
SAB         1
NATI        1
SOUL        1
CH4         1
BNDR        1
SCII        1
MALA        1
HEMI        1
HYUD        1
OTHE        1
DLPH        1
LNRV        1
SUNV        1
WNGO        1
STAR        1
LNGR        1
NOVB        1
FALC        1
HVES        1
LNQT        1
WINE        1
CUSH        1
GILL        1
PRIU        1
EXEC        1
WINB        1
CHEI        1
HOST        1
TETR        1
VECT        1
Name: Make, Length: 532, dtype: int64